In [6]:
import pandas as pd
import pickle
import os

In [7]:
models = 'base'
regression = 'lin'
benchmark_identifier = 'bonner.Object2Vec.regression=lin.regions=EVC,LOC,PFS,OPA,PPA,RSC,FFA,OFA,STS,EBA,visual_degrees=8'
score_dir = '/home/eric/.brainscore_cache/.result_caching/model_tools.brain_transformation.neural.LayerScores._call/'

In [16]:
results = pd.read_csv(f'../results/object2vec_rois_{models}_{regression}.csv')
results.head()

,layer,score,score_error,model,benchmark,effective_dimensionality_dataset,effective_dimensionality_imagenet,model_type,region
0,logits,0.325169,0.004167,base=curvature filters (n=180)-type=engineered...,"bonner.Object2Vec.regression=lin.regions=EVC,L...",12.197068,92.572923,engineered,EVC+LOC+PFS+OPA+PPA+RSC+FFA+OFA+STS+EBA
1,logits,0.266431,0.003997,base=edge filters (n=180)-type=engineered-zsco...,"bonner.Object2Vec.regression=lin.regions=EVC,L...",9.415125,50.055374,engineered,EVC+LOC+PFS+OPA+PPA+RSC+FFA+OFA+STS+EBA
2,logits,0.361042,0.004519,base=random filters (n=180)-type=engineered-zs...,"bonner.Object2Vec.regression=lin.regions=EVC,L...",7.446007,27.578760,engineered,EVC+LOC+PFS+OPA+PPA+RSC+FFA+OFA+STS+EBA
3,logits,0.260202,0.003462,base=curvature filters (n=180)-type=engineered...,"bonner.Object2Vec.regression=lin.regions=EVC,L...",7.058819,7.873873,engineered,EVC+LOC+PFS+OPA+PPA+RSC+FFA+OFA+STS+EBA
4,logits,0.155321,0.003275,base=edge filters (n=180)-type=engineered-1000...,"bonner.Object2Vec.regression=lin.regions=EVC,L...",7.798720,9.538119,engineered,EVC+LOC+PFS+OPA+PPA+RSC+FFA+OFA+STS+EBA


In [9]:
def get_roi_scores(model_identifier):
    score_file = f'model_identifier={model_identifier},benchmark_identifier={benchmark_identifier}.pkl'
    score_file = os.path.join(score_dir, score_file)
    with open(score_file, 'rb') as f:
        score = pickle.load(f)['data']
    score = score.raw.raw
    score = score.groupby('region').mean()
    score = score.mean('subject_id')
    score = score.to_dataframe(name='').reset_index()
    score = score.pivot(index='layer', columns='region', values='')
    score = score.reset_index()
    score.columns.name = None
    score = score.assign(model=model_identifier)
    return score

In [10]:
roi_scores = pd.DataFrame()
for model_identifier in results['model'].unique():
    roi_scores = roi_scores.append(get_roi_scores(model_identifier))

results_with_rois = pd.merge(results, roi_scores, on=['model', 'layer'])
results_with_rois.to_csv(f'../results/object2vec_rois_{models}_{regression}_roiscores.csv', index=False)